# STORY TELLING

In this Jupyter Notebook the data generation for the final project presentation is documented. Find the PDF of my presentation on GitHub: https://github.com/caaarov/finalproject_wildfires_CA.

In [1]:
# Importing necessary libaries

import pandas as pd
import numpy as np
import julian
import datetime

import warnings
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")
warnings.filterwarnings('ignore')
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
pd.set_option('display.max_columns', None)

import re

import h3

In [5]:
df_stations=pd.read_csv("/Users/carolinvogt/Becoming_Data_Analyst/final_project/wildfire_project/CA_weather_stations_clustered_4.csv")
df_stations.drop(["Unnamed: 0"],axis=1, inplace=True)

In [46]:
df_stations

,station_id,latitude,longitude,datacoverage,mindate,maxdate,min_year,max_year,data_period,rad_lng,rad_lat,cluster,h3_level_4,h3_level_3
0,COOP:024299,32.88333,-114.46667,0.9996,1948-01-01,1949-08-01,1948,1949,1,-1.997820,0.573922,-1,84485b5ffffffff,83485bfffffffff
1,COOP:026250,34.15470,-114.29080,0.9755,1931-01-01,2015-11-01,1931,2015,84,-1.994751,0.596112,-1,8429b55ffffffff,8329b5fffffffff
2,COOP:029662,32.73333,-114.61667,0.9906,1948-01-01,1974-04-01,1948,1974,26,-2.000438,0.571304,0,84485bdffffffff,83485bfffffffff
3,COOP:040006,39.03333,-122.43333,0.8618,1960-08-01,1962-12-01,1960,1962,2,-2.136865,0.681260,1,8428339ffffffff,832833fffffffff
4,COOP:040010,38.21770,-121.20130,0.9469,1994-01-01,2015-11-01,1994,2015,21,-2.115362,0.667025,1,8428329ffffffff,832836fffffffff
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4423,WBAN:93242,36.98778,-120.11056,1.0000,2005-01-01,2021-03-04,2005,2021,16,-2.096325,0.645559,7,8429abbffffffff,8329abfffffffff
4424,WBAN:93243,37.23810,-120.88250,1.0000,2004-03-25,2021-03-04,2004,2021,17,-2.109798,0.649927,1,8428369ffffffff,832836fffffffff
4425,WBAN:93244,34.60694,-120.07556,1.0000,2005-08-31,2021-03-04,2005,2021,16,-2.095714,0.604005,7,8429ac9ffffffff,8329acfffffffff
4426,WBAN:93245,38.32080,-123.07470,1.0000,2008-06-14,2021-03-04,2008,2021,13,-2.148059,0.668824,1,842831dffffffff,832831fffffffff


# Temperatur

# Historical data

In [6]:
hexagon_station_dict={}

for row in range(len(df_stations['station_id'])):
    key = df_stations['station_id'][row]
    hexagon_station_dict[key]= df_stations["h3_level_4"][row]
    
len(hexagon_station_dict.keys())

4427

In [23]:
hexagon_station_dict2={}

for row in range(len(df_stations['station_id'])):
    key = df_stations['station_id'][row]
    hexagon_station_dict2[key]= df_stations["h3_level_3"][row]
    
len(hexagon_station_dict2.keys())

4427

In [122]:
df_temp1=pd.read_csv("/Users/carolinvogt/Becoming_Data_Analyst/final_project/wildfire_project/temp_1992_2004.csv")
df_temp1.drop(["Unnamed: 0"],axis=1, inplace=True)

df_temp2=pd.read_csv("/Users/carolinvogt/Becoming_Data_Analyst/final_project/wildfire_project/temp_2004_2016.csv")
df_temp2.drop(["Unnamed: 0"],axis=1, inplace=True)

df_temp3=pd.read_csv("/Users/carolinvogt/Becoming_Data_Analyst/final_project/wildfire_project/temp_2016_2021.csv")
df_temp3.drop(["Unnamed: 0"],axis=1, inplace=True)

df_temp4=pd.read_csv("/Users/carolinvogt/Becoming_Data_Analyst/final_project/wildfire_project/temp_1975_1992.csv")
df_temp4.drop(["Unnamed: 0"],axis=1, inplace=True)

df_temp=pd.concat([df_temp1,df_temp2, df_temp3, df_temp4],ignore_index=True)
df_temp["h3_level_4"]=df_temp["station_id"].apply(lambda x: hexagon_station_dict[x])
df_temp=df_temp.groupby(["h3_level_4","date"]).mean()
df_temp.reset_index(inplace=True)

df_temp=df_temp[df_temp["date"]>"1991-12-01"]
df_temp

,h3_level_4,date,avgTemp
72,8428005ffffffff,1992-01-01,33.3050
73,8428005ffffffff,1992-03-01,33.5894
74,8428005ffffffff,1992-04-01,33.7496
75,8428005ffffffff,1992-05-01,34.8314
76,8428005ffffffff,1992-06-01,34.7162
...,...,...,...
102318,84485bbffffffff,2020-09-01,37.8266
102319,84485bbffffffff,2020-10-01,36.8444
102320,84485bbffffffff,2020-11-01,35.3828
102321,84485bbffffffff,2020-12-01,34.4120


# Forecast with fbProphet

In [123]:
columns={"ds":"date","yhat":"avgTemp","h3_level_4":"h3_level_4"}

In [124]:
df_temp_f=pd.read_csv("/Users/carolinvogt/Becoming_Data_Analyst/final_project/wildfire_project/temp_only_forecast.csv")
df_temp_f.drop(["Unnamed: 0"],axis=1, inplace=True)

df_temp_f=df_temp_f[["ds","yhat","h3_level_4"]]
df_temp_f=df_temp_f.rename(columns=columns)
df_temp_f

,date,avgTemp,h3_level_4
0,2021-01-31,35.015716,8429b55ffffffff
1,2021-02-28,35.245599,8429b55ffffffff
2,2021-03-31,36.026045,8429b55ffffffff
3,2021-04-30,36.817452,8429b55ffffffff
4,2021-05-31,37.834726,8429b55ffffffff
...,...,...,...
12333,2025-08-31,35.152471,8428135ffffffff
12334,2025-09-30,34.571151,8428135ffffffff
12335,2025-10-31,33.535894,8428135ffffffff
12336,2025-11-30,32.826274,8428135ffffffff


In [125]:
df=pd.concat([df_temp,df_temp_f],axis=0)

In [126]:
def join1(row):
    return (row['h3_level_4']+row['date'])

df['join'] = df.apply(join1, axis=1)

In [127]:
df

,h3_level_4,date,avgTemp,join
72,8428005ffffffff,1992-01-01,33.305000,8428005ffffffff1992-01-01
73,8428005ffffffff,1992-03-01,33.589400,8428005ffffffff1992-03-01
74,8428005ffffffff,1992-04-01,33.749600,8428005ffffffff1992-04-01
75,8428005ffffffff,1992-05-01,34.831400,8428005ffffffff1992-05-01
76,8428005ffffffff,1992-06-01,34.716200,8428005ffffffff1992-06-01
...,...,...,...,...
12333,8428135ffffffff,2025-08-31,35.152471,8428135ffffffff2025-08-31
12334,8428135ffffffff,2025-09-30,34.571151,8428135ffffffff2025-09-30
12335,8428135ffffffff,2025-10-31,33.535894,8428135ffffffff2025-10-31
12336,8428135ffffffff,2025-11-30,32.826274,8428135ffffffff2025-11-30


# Percipitation

In [128]:
df_perc1=pd.read_csv("/Users/carolinvogt/Becoming_Data_Analyst/final_project/wildfire_project/perc_1992_1999-11.csv")
df_perc2=pd.read_csv("/Users/carolinvogt/Becoming_Data_Analyst/final_project/wildfire_project/perc_1999-12_2004.csv")
df_perc3=pd.read_csv("/Users/carolinvogt/Becoming_Data_Analyst/final_project/wildfire_project/perc_2004_2012-02.csv")
df_perc4=pd.read_csv("/Users/carolinvogt/Becoming_Data_Analyst/final_project/wildfire_project/perc_2012-03_2016.csv")
df_perc5=pd.read_csv("/Users/carolinvogt/Becoming_Data_Analyst/final_project/wildfire_project/perc_1975_1992.csv")
df_perc6=pd.read_csv("/Users/carolinvogt/Becoming_Data_Analyst/final_project/wildfire_project/perc_2016_2021.csv")


df_perc1.drop(["Unnamed: 0"],axis=1, inplace=True)
df_perc2.drop(["Unnamed: 0"],axis=1, inplace=True)
df_perc3.drop(["Unnamed: 0"],axis=1, inplace=True)
df_perc4.drop(["Unnamed: 0"],axis=1, inplace=True)
df_perc5.drop(["Unnamed: 0"],axis=1, inplace=True)
df_perc6.drop(["Unnamed: 0"],axis=1, inplace=True)

df_perc=pd.concat([df_perc1,df_perc2,df_perc3,df_perc4,df_perc5,df_perc6],ignore_index=True)
df_perc["h3_level_4"]=df_perc["station_id"].apply(lambda x: hexagon_station_dict[x])
df_perc=df_perc.groupby(["h3_level_4","date"]).mean()
df_perc.reset_index(inplace=True)


In [129]:
df_temp_f=pd.read_csv("/Users/carolinvogt/Becoming_Data_Analyst/final_project/wildfire_project/perc_only_forecast.csv")
df_temp_f.drop(["Unnamed: 0"],axis=1, inplace=True)

columns={"ds":"date","yhat":"avgPerc","h3_level_4":"h3_level_4"}
df_temp_f=df_temp_f[["ds","yhat","h3_level_4"]]
df_temp_f=df_temp_f.rename(columns=columns)
#df_temp_f

In [130]:
df2=pd.concat([df_perc,df_temp_f],axis=0)

In [131]:
df2

,h3_level_4,date,avgPerc
0,8428005ffffffff,1975-01-01,197.400000
1,8428005ffffffff,1975-02-01,487.800000
2,8428005ffffffff,1975-03-01,572.100000
3,8428005ffffffff,1975-04-01,104.300000
4,8428005ffffffff,1975-05-01,21.900000
...,...,...,...
10842,8428101ffffffff,2023-11-30,8.881283
10843,8428101ffffffff,2023-12-31,51.652228
10844,8428101ffffffff,2024-01-31,-423.991876
10845,8428101ffffffff,2024-02-29,-14.615525


In [132]:
def join1(row):
    return (row['h3_level_4']+row['date'])

df2['join'] = df2.apply(join1, axis=1)

In [133]:
df2.drop(["h3_level_4","date"],axis=1, inplace=True)

In [134]:
df=pd.merge(df, df2, how="left",on=["join"])
df

,h3_level_4,date,avgTemp,join,avgPerc
0,8428005ffffffff,1992-01-01,33.305000,8428005ffffffff1992-01-01,NaN
1,8428005ffffffff,1992-03-01,33.589400,8428005ffffffff1992-03-01,NaN
2,8428005ffffffff,1992-04-01,33.749600,8428005ffffffff1992-04-01,NaN
3,8428005ffffffff,1992-05-01,34.831400,8428005ffffffff1992-05-01,NaN
4,8428005ffffffff,1992-06-01,34.716200,8428005ffffffff1992-06-01,NaN
...,...,...,...,...,...
81794,8428135ffffffff,2025-08-31,35.152471,8428135ffffffff2025-08-31,14.242964
81795,8428135ffffffff,2025-09-30,34.571151,8428135ffffffff2025-09-30,-30.469897
81796,8428135ffffffff,2025-10-31,33.535894,8428135ffffffff2025-10-31,-7.135579
81797,8428135ffffffff,2025-11-30,32.826274,8428135ffffffff2025-11-30,138.858192


# Wind

In [135]:
df_wind1=pd.read_csv("/Users/carolinvogt/Becoming_Data_Analyst/final_project/wildfire_project/wind_1992_2004.csv")
df_wind2=pd.read_csv("/Users/carolinvogt/Becoming_Data_Analyst/final_project/wildfire_project/wind_2004_2016.csv")
df_wind3=pd.read_csv("/Users/carolinvogt/Becoming_Data_Analyst/final_project/wildfire_project/wind_2016_2021.csv")
df_wind0=pd.read_csv("/Users/carolinvogt/Becoming_Data_Analyst/final_project/wildfire_project/wind_1985_1992.csv")


df_wind1.drop(["Unnamed: 0"],axis=1, inplace=True)
df_wind2.drop(["Unnamed: 0"],axis=1, inplace=True)
df_wind3.drop(["Unnamed: 0"],axis=1, inplace=True)
df_wind0.drop(["Unnamed: 0"],axis=1, inplace=True)

df_wind=pd.concat([df_wind0,df_wind1,df_wind2,df_wind3],ignore_index=True)
df_wind["h3_level_3"]=df_wind["station_id"].apply(lambda x: hexagon_station_dict2[x])
df_wind=df_wind.groupby(["h3_level_3","date"]).mean()
df_wind.reset_index(inplace=True)

df_wind


,h3_level_3,date,avgWind
0,832802fffffffff,1992-04-01,2.7
1,832802fffffffff,1992-05-01,2.8
2,832802fffffffff,1992-06-01,3.1
3,832802fffffffff,1992-07-01,2.9
4,832802fffffffff,1992-08-01,2.6
...,...,...,...
10542,83485bfffffffff,2020-09-01,2.2
10543,83485bfffffffff,2020-10-01,2.4
10544,83485bfffffffff,2020-11-01,2.4
10545,83485bfffffffff,2020-12-01,2.0


In [136]:
hex_l3={}

for row in range(len(df_stations['h3_level_4'])):
    try:
        key = df_stations['h3_level_4'][row]
        hex_l3[key]= df_stations["h3_level_3"][row]
    except:
        continue
len(hex_l3)

247

In [137]:
df_temp_f=pd.read_csv("/Users/carolinvogt/Becoming_Data_Analyst/final_project/wildfire_project/wind_only_forecast.csv")
df_temp_f.drop(["Unnamed: 0"],axis=1, inplace=True)

columns={"ds":"date","yhat":"avgWind","h3_level_4":"h3_level_4"}
df_temp_f=df_temp_f[["ds","yhat","h3_level_4"]]
df_temp_f=df_temp_f.rename(columns=columns)
df_temp_f["h3_level_3"]=df_temp_f["h3_level_4"].apply(lambda x: hex_l3[x])
df_temp_f.drop(["h3_level_4"],axis=1, inplace=True)
df_temp_f

,date,avgWind,h3_level_3
0,2021-01-31,2.466396,8329a5fffffffff
1,2021-02-28,2.949151,8329a5fffffffff
2,2021-03-31,3.290107,8329a5fffffffff
3,2021-04-30,3.233093,8329a5fffffffff
4,2021-05-31,2.934538,8329a5fffffffff
...,...,...,...
3661,2025-08-31,2.036260,832802fffffffff
3662,2025-09-30,1.258233,832802fffffffff
3663,2025-10-31,1.228282,832802fffffffff
3664,2025-11-30,2.693722,832802fffffffff


In [138]:
df3=pd.concat([df_wind,df_temp_f],axis=0)
df3

,h3_level_3,date,avgWind
0,832802fffffffff,1992-04-01,2.700000
1,832802fffffffff,1992-05-01,2.800000
2,832802fffffffff,1992-06-01,3.100000
3,832802fffffffff,1992-07-01,2.900000
4,832802fffffffff,1992-08-01,2.600000
...,...,...,...
3661,832802fffffffff,2025-08-31,2.036260
3662,832802fffffffff,2025-09-30,1.258233
3663,832802fffffffff,2025-10-31,1.228282
3664,832802fffffffff,2025-11-30,2.693722


In [139]:
def join1(row):
    return (row['h3_level_3']+row['date'])

df3['join'] = df3.apply(join1, axis=1)

In [140]:
df3

,h3_level_3,date,avgWind,join
0,832802fffffffff,1992-04-01,2.700000,832802fffffffff1992-04-01
1,832802fffffffff,1992-05-01,2.800000,832802fffffffff1992-05-01
2,832802fffffffff,1992-06-01,3.100000,832802fffffffff1992-06-01
3,832802fffffffff,1992-07-01,2.900000,832802fffffffff1992-07-01
4,832802fffffffff,1992-08-01,2.600000,832802fffffffff1992-08-01
...,...,...,...,...
3661,832802fffffffff,2025-08-31,2.036260,832802fffffffff2025-08-31
3662,832802fffffffff,2025-09-30,1.258233,832802fffffffff2025-09-30
3663,832802fffffffff,2025-10-31,1.228282,832802fffffffff2025-10-31
3664,832802fffffffff,2025-11-30,2.693722,832802fffffffff2025-11-30


In [141]:
df.drop(["join"],axis=1,inplace=True)
df["h3_level_3"]=df["h3_level_4"].apply(lambda x: hex_l3[x])
df['join'] = df.apply(join1, axis=1)
df

,h3_level_4,date,avgTemp,avgPerc,h3_level_3,join
0,8428005ffffffff,1992-01-01,33.305000,NaN,832800fffffffff,832800fffffffff1992-01-01
1,8428005ffffffff,1992-03-01,33.589400,NaN,832800fffffffff,832800fffffffff1992-03-01
2,8428005ffffffff,1992-04-01,33.749600,NaN,832800fffffffff,832800fffffffff1992-04-01
3,8428005ffffffff,1992-05-01,34.831400,NaN,832800fffffffff,832800fffffffff1992-05-01
4,8428005ffffffff,1992-06-01,34.716200,NaN,832800fffffffff,832800fffffffff1992-06-01
...,...,...,...,...,...,...
81794,8428135ffffffff,2025-08-31,35.152471,14.242964,832813fffffffff,832813fffffffff2025-08-31
81795,8428135ffffffff,2025-09-30,34.571151,-30.469897,832813fffffffff,832813fffffffff2025-09-30
81796,8428135ffffffff,2025-10-31,33.535894,-7.135579,832813fffffffff,832813fffffffff2025-10-31
81797,8428135ffffffff,2025-11-30,32.826274,138.858192,832813fffffffff,832813fffffffff2025-11-30


In [142]:
df.drop(["h3_level_3"],axis=1,inplace=True)
df3.drop(["h3_level_3","date"],axis=1,inplace=True)
df_final=pd.merge(df, df3, how="left",on=["join"])

In [143]:
df_final.isna().sum()

h3_level_4        0
date              0
avgTemp           0
avgPerc       14968
join              0
avgWind       25073
dtype: int64

In [144]:
df_final = df_final[df_final['avgWind'].notna()]
df_final = df_final[df_final['avgPerc'].notna()]

In [145]:
df_final.drop(["join"],axis=1,inplace=True)

In [146]:
df_final.shape

(58812, 5)

In [147]:
df_final.isna().sum()

h3_level_4    0
date          0
avgTemp       0
avgPerc       0
avgWind       0
dtype: int64

# Adding features

In [148]:
def join_quarter(row):
    return (row['h3_level_4']+str(row['quarter'])+str(row['year']))

In [149]:
def avg_temp(df_temp,column_name):
    df_temp1=df_temp
    
    df_temp['date'] = pd.to_datetime(df_temp['date'])
    df_temp["quarter"]=df_temp["date"].apply(lambda x: x.quarter)
    df_temp["year"]=df_temp["date"].apply(lambda x: x.year)
    df_temp['join']=df_temp.apply(join_quarter, axis=1)
    
    df_temp1['date'] = pd.to_datetime(df_temp1['date'])
    df_temp1["quarter"]=df_temp1["date"].apply(lambda x: x.quarter)
    df_temp1["year"]=df_temp1["date"].apply(lambda x: x.year)
    df_temp1=df_temp1.groupby(["h3_level_4","quarter","year"]).mean()
    df_temp1.reset_index(inplace=True)
    
    df_temp1['join']=df_temp1.apply(join_quarter, axis=1)
    
    quarter_dic={}
    
    for row in range(len(df_temp1['join'])):
        key = df_temp1['join'][row]
        quarter_dic[key]= df_temp1["avgTemp"][row]
        
    df_temp[column_name]=df_temp['join'].apply(lambda x: quarter_dic[x])
    df_temp.drop(["join","quarter","year"],axis=1, inplace=True)
    #df_temp['date'] = df_temp['date'].apply(lambda x: (str(x)[:11]))
    
    return df_temp

In [150]:
df_final=avg_temp(df_final,"q_avg_temp")

In [151]:
def join_year(row):
    return (row['h3_level_4']+str(row['year']))

In [152]:
def avg_temp_year(df_temp,column_name):
    df_temp1=df_temp
    
    df_temp['date'] = pd.to_datetime(df_temp['date'])
    df_temp["year"]=df_temp["date"].apply(lambda x: x.year)
    df_temp['join']=df_temp.apply(join_year, axis=1)
    
    df_temp1['date'] = pd.to_datetime(df_temp1['date'])
    df_temp1["year"]=df_temp1["date"].apply(lambda x: x.year)
    df_temp1=df_temp1.groupby(["h3_level_4","year"]).mean()
    df_temp1.reset_index(inplace=True)
    
    df_temp1['join']=df_temp1.apply(join_year, axis=1)
    
    quarter_dic={}
    
    for row in range(len(df_temp1['join'])):
        key = df_temp1['join'][row]
        quarter_dic[key]= df_temp1["avgTemp"][row]
        
    df_temp[column_name]=df_temp['join'].apply(lambda x: quarter_dic[x])
    df_temp.drop(["join","year"],axis=1, inplace=True)
    #df_temp['date'] = df_temp['date'].apply(lambda x: (str(x)[:11]))
    
    return df_temp

In [153]:
df_final=avg_temp_year(df_final,"y_avg_temp")

In [155]:
df_final["avgPerc"]=df_final["avgPerc"].apply(lambda x: x if x > 0 else 0)

In [156]:
df_final

,h3_level_4,date,avgTemp,avgPerc,avgWind,q_avg_temp,y_avg_temp
335,8428021ffffffff,1992-05-01,35.479400,4.800000,2.800000,35.668400,35.727800
336,8428021ffffffff,1992-06-01,35.857400,29.300000,3.100000,35.668400,35.727800
337,8428021ffffffff,1992-07-01,36.219200,17.500000,2.900000,36.050600,35.727800
338,8428021ffffffff,1992-08-01,36.280400,0.000000,2.600000,36.050600,35.727800
339,8428021ffffffff,1992-09-01,35.652200,1.800000,2.900000,36.050600,35.727800
...,...,...,...,...,...,...,...
91820,8429a8dffffffff,2024-06-30,36.768345,0.000000,2.960966,35.904207,35.361974
91821,8429a8dffffffff,2024-07-31,36.362008,1.449372,3.035301,36.243811,35.361974
91822,8429a8dffffffff,2024-07-31,36.362008,1.449372,2.849477,36.243811,35.361974
91823,8429a8dffffffff,2024-08-31,36.125613,3.956220,2.897089,36.243811,35.361974


In [157]:
def avg_perc(df_temp,column_name):
    df_temp1=df_temp
    
    df_temp['date'] = pd.to_datetime(df_temp['date'])
    df_temp["quarter"]=df_temp["date"].apply(lambda x: x.quarter)
    df_temp["year"]=df_temp["date"].apply(lambda x: x.year)
    df_temp['join']=df_temp.apply(join_quarter, axis=1)
    
    df_temp1['date'] = pd.to_datetime(df_temp1['date'])
    df_temp1["quarter"]=df_temp1["date"].apply(lambda x: x.quarter)
    df_temp1["year"]=df_temp1["date"].apply(lambda x: x.year)
    df_temp1=df_temp1.groupby(["h3_level_4","quarter","year"]).sum()
    df_temp1.reset_index(inplace=True)
    
    df_temp1['join']=df_temp1.apply(join_quarter, axis=1)
    
    quarter_dic={}
    
    for row in range(len(df_temp1['join'])):
        key = df_temp1['join'][row]
        quarter_dic[key]= df_temp1["avgPerc"][row]
        
    df_temp[column_name]=df_temp['join'].apply(lambda x: quarter_dic[x])
    df_temp.drop(["join","quarter","year"],axis=1, inplace=True)
    #df_temp['date'] = df_temp['date'].apply(lambda x: (str(x)[:11]))
    
    return df_temp

In [158]:
df_final=avg_perc(df_final,"q_sum_perc")

In [159]:
def avg_temp_year(df_temp,column_name):
    df_temp1=df_temp
    
    df_temp['date'] = pd.to_datetime(df_temp['date'])
    df_temp["year"]=df_temp["date"].apply(lambda x: x.year)
    df_temp['join']=df_temp.apply(join_year, axis=1)
    
    df_temp1['date'] = pd.to_datetime(df_temp1['date'])
    df_temp1["year"]=df_temp1["date"].apply(lambda x: x.year)
    df_temp1=df_temp1.groupby(["h3_level_4","year"]).sum()
    df_temp1.reset_index(inplace=True)
    
    df_temp1['join']=df_temp1.apply(join_year, axis=1)
    
    quarter_dic={}
    
    for row in range(len(df_temp1['join'])):
        key = df_temp1['join'][row]
        quarter_dic[key]= df_temp1["avgPerc"][row]
        
    df_temp[column_name]=df_temp['join'].apply(lambda x: quarter_dic[x])
    df_temp.drop(["join","year"],axis=1, inplace=True)
    #df_temp['date'] = df_temp['date'].apply(lambda x: (str(x)[:11]))
    
    return df_temp

In [160]:
df_final=avg_temp_year(df_final,"y_sum_perc")

In [162]:
df_final["lat"]=df_final["h3_level_4"].apply(lambda x: h3.h3_to_geo(x)[0])
df_final["lng"]=df_final["h3_level_4"].apply(lambda x: h3.h3_to_geo(x)[1])
df_final

,h3_level_4,date,avgTemp,avgPerc,avgWind,q_avg_temp,y_avg_temp,q_sum_perc,y_sum_perc,lat,lng
335,8428021ffffffff,1992-05-01,35.479400,4.800000,2.800000,35.668400,35.727800,34.100000,138.300000,40.779448,-123.587920
336,8428021ffffffff,1992-06-01,35.857400,29.300000,3.100000,35.668400,35.727800,34.100000,138.300000,40.779448,-123.587920
337,8428021ffffffff,1992-07-01,36.219200,17.500000,2.900000,36.050600,35.727800,19.300000,138.300000,40.779448,-123.587920
338,8428021ffffffff,1992-08-01,36.280400,0.000000,2.600000,36.050600,35.727800,19.300000,138.300000,40.779448,-123.587920
339,8428021ffffffff,1992-09-01,35.652200,1.800000,2.900000,36.050600,35.727800,19.300000,138.300000,40.779448,-123.587920
...,...,...,...,...,...,...,...,...,...,...,...
91820,8429a8dffffffff,2024-06-30,36.768345,0.000000,2.960966,35.904207,35.361974,3.256989,213.866672,35.832715,-119.335939
91821,8429a8dffffffff,2024-07-31,36.362008,1.449372,3.035301,36.243811,35.361974,10.811184,213.866672,35.832715,-119.335939
91822,8429a8dffffffff,2024-07-31,36.362008,1.449372,2.849477,36.243811,35.361974,10.811184,213.866672,35.832715,-119.335939
91823,8429a8dffffffff,2024-08-31,36.125613,3.956220,2.897089,36.243811,35.361974,10.811184,213.866672,35.832715,-119.335939


In [163]:
df_final.to_csv("/Users/carolinvogt/Becoming_Data_Analyst/final_project/wildfire_project/UC_database.csv")

print("Sucessfully saved")

Sucessfully saved
